In [2]:
import pandas as pd
import numpy as np

### Combining NGS Data by Year

In [ ]:
# 2016 Season Data Processing
ngs_2016_pre = pd.read_csv('data/NGS-2016-pre.csv')
ngs_2016_1_6 = pd.read_csv('data/NGS-2016-reg-wk1-6.csv')
ngs_2016_7_12 = pd.read_csv('data/NGS-2016-reg-wk7-12.csv')
ngs_2016_13_17 = pd.read_csv('data/NGS-2016-reg-wk13-17.csv')
ngs_2016_post = pd.read_csv('data/NGS-2016-post.csv')

# Combine
ngs_2016 = pd.concat([ngs_2016_pre, ngs_2016_1_6, ngs_2016_7_12, ngs_2016_13_17, ngs_2016_post], axis=0)
ngs_2016.to_csv('data/NGS-2016.csv', index=False)

# Clear up memory
del ngs_2016_pre
del ngs_2016_1_6
del ngs_2016_7_12
del ngs_2016_13_17
del ngs_2016_post

# 2017 Season Data Processing
ngs_2017_pre = pd.read_csv('data/NGS-2017-pre.csv')
ngs_2017_1_6 = pd.read_csv('data/NGS-2017-reg-wk1-6.csv')
ngs_2017_7_12 = pd.read_csv('data/NGS-2017-reg-wk7-12.csv')
ngs_2017_13_17 = pd.read_csv('data/NGS-2017-reg-wk13-17.csv')
ngs_2017_post = pd.read_csv('data/NGS-2017-post.csv')

# Combine
ngs_2017 = pd.concat([ngs_2017_pre, ngs_2017_1_6, ngs_2017_7_12, ngs_2017_13_17, ngs_2017_post], axis=0)
ngs_2017.to_csv('data/NGS-2017.csv', index=False)

# Clear up memory
del ngs_2017_pre
del ngs_2017_1_6
del ngs_2017_7_12
del ngs_2017_13_17
del ngs_2017_post

### Combining NGS Data with Concussion Data

In [ ]:
# Load in all NGS data and punt_return play info
ngs_2016 = pd.read_csv('data/NGS-2016.csv')
ngs_2017 = pd.read_csv('data/NGS-2017.csv')
concussion_df = pd.read_csv('data/video_review.csv')
remainder_df = concussion_df.groupby(['GameKey','PlayID']).size().reset_index().rename(columns={0:'count'})

In [ ]:
# Create condensed set of NGS data
condensed_ngs_2016 = pd.merge(remainder_df, ngs_2016,
                          how='inner',
                          on=['GameKey', 'PlayID'])

# Memory Management
del ngs_2016

condensed_ngs_2017 = pd.merge(remainder_df, ngs_2017,
                          how='inner',
                          on=['GameKey', 'PlayID'])

# Memory Management
del ngs_2017
del remainder_df

condensed_ngs = pd.concat([condensed_ngs_2016, condensed_ngs_2017], axis=0)

# Memory Management
del condensed_ngs_2016
del condensed_ngs_2017

In [ ]:
# Combine Player Roles with NGS Data
play_player_role_df = pd.read_csv('data/play_player_role_data.csv')
play_df = pd.read_csv('data/play_information.csv')

role_ngs = pd.merge(condensed_ngs, play_player_role_df,
                    how='left',
                    on=['GameKey', 'PlayID', 'GSISID'])

role_ngs = pd.merge(role_ngs, play_df,
                    how='left',
                    on=['GameKey', 'PlayID'])

# Cleanup
keepers = ['GameKey', 'PlayID', 'GSISID', 'Time', 'x', 'y', 'dis', 'o', 'Event', 'Role', 'PlayDescription']
role_ngs = role_ngs[keepers]

# Memory Management
del play_player_role_df
del condensed_ngs

role_ngs.to_csv('data/NGS-concussions.csv', index=False)

### Combining NGS Data with Punt Returns
- We'll be taking our condensed set of games and natural joining with the NGS data for each year and then recombining data so we have a dataset of 'cleanish' punt returns. We'll use this data to get conditional statistics on returns given distance of closest punt player to punt receiver. We use the assumption that all the punt receiver is always the one who is getting the ball at the 'Event': 'punt_returned' from the NGS data.
- 'play-punt_returns.csv' comes from preprocessing notebook
- **LINK TO PUNT RETURN NOTEBOOK**

In [ ]:
# Load in all NGS data and punt_return play info
ngs_2016 = pd.read_csv('data/NGS-2016.csv')
ngs_2017 = pd.read_csv('data/NGS-2017.csv')
remainder_df = pd.read_csv('data/play-punt_returns.csv')

In [ ]:
# Create condensed set of NGS data
condensed_ngs_2016 = pd.merge(remainder_df, ngs_2016,
                          how='inner',
                          on=['GameKey', 'PlayID'])

# Memory Management
del ngs_2016

condensed_ngs_2017 = pd.merge(remainder_df, ngs_2017,
                          how='inner',
                          on=['GameKey', 'PlayID'])

# Memory Management
del ngs_2017
del remainder_df

condensed_ngs = pd.concat([condensed_ngs_2016, condensed_ngs_2017], axis=0)

# Memory Management
del condensed_ngs_2016
del condensed_ngs_2017

In [ ]:
# Combine Player Roles with NGS Data
play_player_role_df = pd.read_csv('data/play_player_role_data.csv')

role_ngs = pd.merge(condensed_ngs, play_player_role_df,
                    how='left',
                    on=['GameKey', 'PlayID', 'GSISID'])

droppers = ['Season_Year_x', 'Season_Year_y']
role_ngs.drop(columns=droppers, inplace=True)

# Memory Management
del play_player_role_df
del condensed_ngs

role_ngs.to_csv('data/NGS-punt_returns.csv', index=False)

### Combining NGS Data with Fair Catches
- We combine this data to calculate proximity data for these plays which will help to build an inuition of when a fair catch occurs relative to the punt teams positioning.
- 'play-fc.csv' comes from the preproessing notebook **LNIK TO NOTEBOOKJ**
- **LINK TO FAIR CATCH ANALYSIS NOTEBOOK**

In [ ]:
# Load in all NGS data and punt_return play info
ngs_2016 = pd.read_csv('data/NGS-2016.csv')
ngs_2017 = pd.read_csv('data/NGS-2017.csv')
remainder_df = pd.read_csv('data/play-fc.csv')

In [ ]:
ngs

In [ ]:
# Create condensed set of NGS data
condensed_ngs_2016 = pd.merge(remainder_df, ngs_2016,
                          how='inner',
                          on=['GameKey', 'PlayID'])

# Memory Management
del ngs_2016

condensed_ngs_2017 = pd.merge(remainder_df, ngs_2017,
                          how='inner',
                          on=['GameKey', 'PlayID'])

# Memory Management
del ngs_2017
del remainder_df

condensed_ngs = pd.concat([condensed_ngs_2016, condensed_ngs_2017], axis=0)

# Memory Management
del condensed_ngs_2016
del condensed_ngs_2017

In [ ]:
# Combine Player Roles with NGS Data
play_player_role_df = pd.read_csv('data/play_player_role_data.csv')

role_ngs = pd.merge(condensed_ngs, play_player_role_df,
                    how='left',
                    on=['GameKey', 'PlayID', 'GSISID'])

droppers = ['Season_Year_x', 'Season_Year_y']
role_ngs.drop(columns=droppers, inplace=True)

# Memory Management
del play_player_role_df
del condensed_ngs

role_ngs.to_csv('data/NGS-fc.csv', index=False)

In [ ]:
role_ngs.shape

### ALTERNATIVE PROCESSING

In [3]:
%%time

# 2016 Season Data Processing
ngs_2016_pre = pd.read_csv('data/NGS-2016-pre.csv')
ngs_2016_1_6 = pd.read_csv('data/NGS-2016-reg-wk1-6.csv')
ngs_2016_7_12 = pd.read_csv('data/NGS-2016-reg-wk7-12.csv')
ngs_2016_13_17 = pd.read_csv('data/NGS-2016-reg-wk13-17.csv')
ngs_2016_post = pd.read_csv('data/NGS-2016-post.csv')

# Combine
ngs_2016 = pd.concat([ngs_2016_pre, ngs_2016_1_6, ngs_2016_7_12, ngs_2016_13_17, ngs_2016_post], axis=0)

# Clear up memory
del ngs_2016_pre
del ngs_2016_1_6
del ngs_2016_7_12
del ngs_2016_13_17
del ngs_2016_post

# 2017 Season Data Processing
ngs_2017_pre = pd.read_csv('data/NGS-2017-pre.csv')
ngs_2017_1_6 = pd.read_csv('data/NGS-2017-reg-wk1-6.csv')
ngs_2017_7_12 = pd.read_csv('data/NGS-2017-reg-wk7-12.csv')
ngs_2017_13_17 = pd.read_csv('data/NGS-2017-reg-wk13-17.csv')
ngs_2017_post = pd.read_csv('data/NGS-2017-post.csv')

# Combine
ngs_2017 = pd.concat([ngs_2017_pre, ngs_2017_1_6, ngs_2017_7_12, ngs_2017_13_17, ngs_2017_post], axis=0)

# Clear up memory
del ngs_2017_pre
del ngs_2017_1_6
del ngs_2017_7_12
del ngs_2017_13_17
del ngs_2017_post

# Combine
ngs_all = pd.concat([ngs_2016, ngs_2017], axis=0)

# Clear up memory
del ngs_2016
del ngs_2017

# Drop unneeded columns
droppers = ['Season_Year', 'o', 'dir']
ngs_all.drop(columns=droppers, inplace=True)

<string>:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 2min 36s, sys: 41.8 s, total: 3min 18s
Wall time: 2min 48s


- Get NGS subsets

In [10]:
# Fair Catch
fair_catch_df = pd.read_csv('data/play-fair_catch.csv')
remainder_df = fair_catch_df.groupby(['GameKey','PlayID']).size().reset_index().rename(columns={0:'count'})

# Create condensed set of NGS data
condensed_ngs = pd.merge(remainder_df, ngs_all,
                          how='inner',
                          on=['GameKey', 'PlayID'])

condensed_ngs.to_csv('data/NGS-fair_catch.csv', index=False)

In [12]:
# Punt Return
fair_catch_df = pd.read_csv('data/play-punt_return.csv')
remainder_df = fair_catch_df.groupby(['GameKey','PlayID']).size().reset_index().rename(columns={0:'count'})

# Create condensed set of NGS data
condensed_ngs = pd.merge(remainder_df, ngs_all,
                          how='inner',
                          on=['GameKey', 'PlayID'])

condensed_ngs.to_csv('data/NGS-punt_return.csv', index=False)

In [13]:
# Concussion
concussion_df = pd.read_csv('data/video_review.csv')
remainder_df = concussion_df.groupby(['GameKey','PlayID']).size().reset_index().rename(columns={0:'count'})

# Create condensed set of NGS data
condensed_ngs = pd.merge(remainder_df, ngs_all,
                          how='inner',
                          on=['GameKey', 'PlayID'])

condensed_ngs.to_csv('data/NGS-concussion.csv', index=False)

### Combine NGS data with concussion data

In [ ]:
# Combine NGS data with concussion data
concussion_df = pd.read_csv('data/video_review.csv')
remainder_df = concussion_df.groupby(['GameKey','PlayID']).size().reset_index().rename(columns={0:'count'})

del concussion_df

# Create condensed set of NGS data
condensed_ngs = pd.merge(remainder_df, ngs_all,
                          how='inner',
                          on=['GameKey', 'PlayID'])

del remainder_df

# Combine Player Roles with NGS Data
play_player_role_df = pd.read_csv('data/play_player_role_data.csv')
play_df = pd.read_csv('data/play_information.csv')

role_ngs = pd.merge(condensed_ngs, play_player_role_df,
                    how='left',
                    on=['GameKey', 'PlayID', 'GSISID'])

role_ngs = pd.merge(role_ngs, play_df,
                    how='left',
                    on=['GameKey', 'PlayID'])

# Cleanup
keepers = ['GameKey', 'PlayID', 'GSISID', 'Time', 'x', 'y', 'dis', 'Event', 'Role', 'PlayDescription']
role_ngs = role_ngs[keepers]

role_ngs.to_csv('NGS-concussions.csv', index=False)

del condensed_ngs
del play_df
del role_ngs